# AI Homework 3: Comparing Search Algorithms
Stan Slupecki

## Introduction

One of the cornerstones of AI is using search algorithms. Here we compare two search algorithms: iterative deepening and a* search. In this project there effectiveness is measured by how efficiently they solve an 8-block sliding puzzle.

Iterative deepening builds on another search algorithm, depth-limited search. Depth limited itself derives from depth first search, a search algorithm where the deepest possible solutions are explored first. Depth limited search's strength is it uses very limited system memory. However, it is a risky strategy for two reasons. First, the first solution found is probably not the most efficient solution. Since this explores the deepest solution first, and shallow solutions are more efficient, the first solution found won't likely be a shallow one. The second problem with a depth first search is difficulty determining an end point. Depth limited search fixes this by performing a depth first search that will stop searching at specified solution depth. It can return a default value if needed. Iterative deepening runs a depth first search repeatedly until a solution is found, incrementally increasing the depth the depth limited search, and re-running the search. This guarantees a solution with a depth limited search, but also ensures it will be found at the shallowest possible depth.

A\* is a varient of Dijkstra's algorithm. Dijkstra's algorithm computes the shortest path between two nodes in a graph. Starting at the starting node, a node is selected, and the cost of the path to each of the nodes neighbors is calculated and each neighboring node is stored in a priority queue. Priority in the queue is defined such that the nodes with shorter paths are stored first in the queue. The first element in this queue is popped, and it's neighbor's paths are computed and stored in the queue. This process continues in this manner until the end node is reached, and a result is returned. More for when the algorithm is used in search algorithms, if the queue is empty, then the search terminates. A\* builds on this by using a heuristic that estimates how close the current state is to the end state, and adding it to the path cost before storing the node in the priority queue. Assuming the heuristic function is well designed, a\* has a reputation for being very efficient.

## Discusion of solution

For both problems, I decided to use the pre-made AIMA Python code in conjunction with the example code supplied with the homework assignment.

The AIMA search functions require an object who's class inherits from AIMA's Problem class. This Problem class models the problem you're trying to solve, in this case, the slider puzzle. The advantage of using AIMA's Problem class is, assuming the child class properly models the problem, the same Problem object can be passed to any of AIMA's search functions without modification. This greatly simplifies comparing two search functions, as the problem model is the same for all AIMA search functions. To use implement a child class of the Problem class, I had to override the following virtual methods (besides the \__init\__ function): actions, result, and goal_test.

Actions is a method that is passed a state, and generates an iterable of legal actions that can be performed in the given state. Since I wanted to use the premade example code, which doesn't split up finding actions from generating new solutions, I iterated over a list of possible directions and used the example code to generate a new solution in that direction. If a new valid solution was generated, then that direction was yielded as a valid action.

Result takes a state and a legal action, and generates a new state representing a possible solution. In this case, state represented a 8-puzzle board configuration, and action represented a possible direction that a piece could be moved to. I chose to use the apply_action() function supplied in the example code. This takes a board configuration and a direction and moves the empty slot of the 8-puzzle in that direction.

The goal_test method is passed the current state and returns true if the goal state has been reached. I passed the state, which represents a board configuration, to the goal_test() function supplied in the example code. This checks if the slider puzzle is solved, and returns true if it has been, false otherwise.

## Testing Design and Expected Results

### Testing Design

Each search algorithm was tested with the same methods, though the setup for the a* search was slightly more involved. 

First, each search algorithm is presented with a randomly generated 8-puzzle to solve.

Both search algorithms were tested with Jupyter's timeit magic command. This is a utility built into Jupyter that will compute the mean execution time of a function. In this case, I specified for 3 runs of each search function using the "-r 3" option. This returns time in seconds and milliseconds, the main metric of this project.

Lastly, the a\* search required a heuristic function. I passed a function that calculates "Manhattan Distance" for the tiles. The Manhattan Distance function computes the sum of the horizontal and vertical distance between a tile's current position and its intended position, then sums the value for each tile into one value. This value is used to estimate how close the current state is to the goal state. This value is added to the current node's path length to create an estimate of how efficient the current path is.

### Expected Results

I expect a* to outperform iterative deepening.

Iterative deepening is a simpler algorithm. It's main advantage is it doesn't store much data when peforming a search. This makes it a good option for simpler systems with limited system resources. However, it is quite capable of generating overly complicated solutions, if it finds one at all.

A\* on the other hand can be very efficient if an effective heuristic is used. Because a\* constantly estimates the efficiency of the current path and actively swaps to more efficient paths when presented with the chance, it is much more active in pursuiting the end goal than iterative deepening. This is, of course, dependent on the heuristic used. I used the Manhattan Distance heuristic, a well established, admissable heuristic. That is, it never overestimates the final path cost of the solution path, meaning it won't accidentally mark an efficient solution as inefficient. As such, I feel comfortable in saying it will outperform iterative deepening.

## Test

### Setup

In [1]:
# this allows the python files in the project directory to be imported
import sys
sys.path.append('/home/nbuser/library')

from math import floor
from search import iterative_deepening_search, astar_search
import eight_puzzle
import numpy as np
import timeit

# Used to print the solution found by a search function
def print_path(node):
    if node.parent:
        print_path(node.parent)
    print("Step: " + str(node.depth + 1))
    print(node.state[0])
    print(node.state[1])
    print(node.state[2])
    
# Create a problem object to model an eight puzzle
problem = eight_puzzle.EightPuzzleProblem()

## A* Search

In [2]:
# The heuristic used is Manhattan distance, a sum of the vertical and horizontal distance
# between a slider tile's current position and its proper position
def manhattan_distance(state):
    side_length = len(state)
    # find the length of the side of the puzzle

    distance = 0
    # the manhattan distance
    for i in range(side_length):
        for j in range(side_length):
            piece = state[i][j]
            if piece != 0:
                # find the intended row and column of the current piece
                original_row = floor((piece-1)/side_length)
                original_column = (piece-1) % side_length

                # add the current manhattan distance to a cumulative value
                distance += abs(i - original_row) + abs(j - original_column)
    return distance

In [3]:
# run a* three times to get an average runtime
%timeit -r 3 astar_search(problem, lambda n: manhattan_distance(n.state))

2.71 s ± 85.9 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [4]:
# this is to demonstrate what a solution looks like
 print_path(astar_search(problem, lambda n: manhattan_distance(n.state)))

Step: 1
(0, 1, 2)
(3, 4, 5)
(6, 7, 8)
Step: 2
(1, 0, 2)
(3, 4, 5)
(6, 7, 8)
Step: 3
(1, 4, 2)
(3, 0, 5)
(6, 7, 8)
Step: 4
(1, 4, 2)
(0, 3, 5)
(6, 7, 8)
Step: 5
(1, 4, 2)
(5, 3, 0)
(6, 7, 8)
Step: 6
(1, 4, 2)
(5, 0, 3)
(6, 7, 8)
Step: 7
(1, 0, 2)
(5, 4, 3)
(6, 7, 8)
Step: 8
(1, 2, 0)
(5, 4, 3)
(6, 7, 8)
Step: 9
(1, 2, 3)
(5, 4, 0)
(6, 7, 8)
Step: 10
(1, 2, 3)
(5, 0, 4)
(6, 7, 8)
Step: 11
(1, 2, 3)
(0, 5, 4)
(6, 7, 8)
Step: 12
(1, 2, 3)
(6, 5, 4)
(0, 7, 8)
Step: 13
(1, 2, 3)
(6, 5, 4)
(7, 0, 8)
Step: 14
(1, 2, 3)
(6, 0, 4)
(7, 5, 8)
Step: 15
(1, 2, 3)
(0, 6, 4)
(7, 5, 8)
Step: 16
(1, 2, 3)
(4, 6, 0)
(7, 5, 8)
Step: 17
(1, 2, 3)
(4, 0, 6)
(7, 5, 8)
Step: 18
(1, 2, 3)
(4, 5, 6)
(7, 0, 8)
Step: 19
(1, 2, 3)
(4, 5, 6)
(7, 8, 0)


## Iterative Deepening

In [ ]:
%timeit -r 3 iterative_deepening_search(problem)

In [ ]:
# this is to demonstrate what a solution looks like
 print_path(iterative_deepening_search(problem))

## Actual Results

| Search Algorithm | Results  |
|---|---|
| Iterative Deepening | n/a  |
| A* Search| 2.71 s ± 124 ms |

Note that because the initial puzzle configuration is randomly generated, the average time taken to search for a solution may vary slightly.

In addition, iterative deepening can take a very long time to generate results. I tested my code and AIMA's provided solution for the 8-puzzle (which is genrally more efficient than my solution), and found that it could take around 30 minutes to return results, depending on the puzzle. Currently, the iterative deepening function has been running over two hours in this notebook, and still has yet to return a result.

## Discussion of Results

The results are in line with my prediction. A* returned a value in about 3 seconds, while over two hours later, the iterative deepening search is still running.

## Conclusion

In conclusion, a\* is a highly effective search algorithm. While iterative deepening can be usefull when system resources are limited, it appears for a slider puzzle, this particular search algorithm doesn't work well. Given how the iterative deepening has been running for over two hours, there is even the chance the solution graphs it is currently generating occupy as much if not more memory than a\*, without returning any results in a timely manner. Given that there are variants of a\* that use even less system resources, I would consider using them before using iterative deepening for finding a solution to a search problem.